In [ ]:
# Testing Cell

# Converting from legacy `phase_info` user options

If you are using a version of Aviary that was released before 0.10.0, then you may have some aircraft phase_info files that were written before a major reorganization of the options. You will need to convert them to the new format, and this document is meant to provide some guidance. The redesign of the phase options was driven by the following goals:

* Provide a set of keys that is more consistent across the different phase types.
* Remove redundancies, particularly cases where we can replace 2 keys with 1.
* Rename keys to be more understandable to the user.
* Add missing options such as certain missing scaling parameters.


## 